In [5]:
# import packages
import IPython
import ipywidgets
import ipyleaflet
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import numpy as np
import random
import os
import tensorflow as tf

import descarteslabs as dl
import descarteslabs.workflows as wf

### Create Geocontext (Currently not actually using any of this except the dates)
I was originally aiming to only compute the resulting mosaic over the AOI only using the geocontext but I binned it in the end and just let the interactive map constrain it to the AOI. Might be worth looking into further

In [6]:
with open("aoi_small_train.geojson") as f:
    fc = json.load(f)
    
aoi_geo = fc["features"][0]["geometry"]

aoi = wf.GeoContext(
    geometry = aoi_geo,
    crs="EPSG:3857",
    resolution = 10.0)

start_datetime = "2020-04-01"
end_datetime = "2020-07-28"

### Sentinel 2 stack w/ cloud mask
Using whatever bands you like

In [7]:
sentinel_stack = (wf.ImageCollection.from_id("sentinel-2:L1C",
                                            start_datetime = start_datetime,
                                            end_datetime = end_datetime)
                  .pick_bands("red green blue cloud-mask red-edge-2 red-edge-3 red-edge-4 nir swir1 swir2")
                 )

sentinel_stack = sentinel_stack.filter(lambda img:img.properties["cloud_fraction"] <= 0.06)

sentinel_masked = sentinel_stack.map(lambda img: img.mask(img.pick_bands('cloud-mask')==1))

### Create the image mosaic and compute over the AOI 

In [8]:
sentinel_mosaic = (sentinel_masked
                   .mosaic()
                   .pick_bands("red green blue"))

#sentinel_image = sentinel_mosaic.compute(aoi)

## Visualise the image
I think it looks pretty good, but there's room for improvement. Zoom out to see where it's not great (outside the city)

In [9]:
a_map = wf.interactive.MapApp()
a_map.center = (18.5265, -69.9129)
a_map.zoom = 15

sentinel_mosaic.visualize('image', scales=[(0, 1), (0, 1), (0, 1)], map=a_map)

a_map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.
